# Homework Set - III

Each Question/subquestion is worth **5 points**, unless indicated otherwise.

### ENCODE: Testing for Enrichment

Let's try to learn about possible genetic mechanisms of Alzheimer's using ENCODE data.

SNPs associated with this disease can be found in `ALZ_SNPs_hg38.bed`.

Choose an ENCODE data set from the ENCODE website that you think may be relevant to the disease (note: feel free to choose a tissue other than brain...there could be other tissues linked to the disease as well). Think both about tissue-type and ENCODE mark. Please do not use DNAse hypersensitivity or RNA-seq datasets. 

**Hint: it will be easiest if you find a data set with data in the bed file format.**

**Note: Make sure the coordinates of the file you pick also use the hg38 reference genome**

**Note: PLEASE try to pick a file size that is Less than 1 Gb -- this will make it run faster/Feasibly on CoCalc!**

**Q1.** (2 points) What dataset did you choose and why? Include the tissue, epigenetic mark tested, and any identifying information, such as the name of the sample.

**Q2:** (5 points) Now, test for significant overlap bewtween the Alzheimer's SNPs and your ENCODE mark. This time, instead of generating fake SNP sets, use the fisher exact test implemented in bedtools (http://bedtools.readthedocs.org/en/latest/content/tools/fisher.html). 

**NOTE: The fisher tool requires that your data is pre-sorted by chromosome and then by start position. e.g.**

```bash
$ sort -k1,1 -k2,2n in.bed > in.sorted.bed 
```
    
**for BED files, where `in.bed` is your input bed file and `in.sorted.bed` is the name that you want for the sorted output.**

This test looks for an association between two classifications. In our case, we are looking for an association between being a SNP in ALZ_SNPs_hg38.bed and being in an interval from your ENCODE data set. Run this test on your datasets. 

**NOTE: We have provided you a pre-sorted genome file in this directory that you can use for this analysis (human.hg38.genome).**

What is the two-tailed p-value? This is the p-value that under the null hypothesis (random chance), the given parameter (probability of overlap) is more extreme than what we observe in our data. Please include your unix commands in the top box and your answer in the bottom box.

**Q3.** (5 points)  Was there a significant association between the disease-associated genetic variants and the ENCODE mark? Explain your findings to the best of your abilities. Are you suprised by the result? If so, why? If not, why not?

### ENCODE: Finding Active regulatory regions

You also know that H3K9me3 is a sign of *repressed* chromatin. Download and unzip the narrowPeak bed file for mouse H3K9me3 data (file **ENCFF234GVB**). Then, using the intersect command and -v flag, count the number of genes from MeAc_genes.bed that do **NOT** also have a H3K9me3 mark. MeAc_genes.bed was generated in the 2nd Encode Module and has been provided in this folder.

**Q4.** (5 points) Write the bedtools command(s) to do this.

**Q5.** (5 points) How many genes from MeAc_genes.bed do not have a H3K9me3 mark?

**Q6.** (5 points) Are you convinced that these genes are truly active genes in embroynic mouse liver? Do you think that there may be some genes in liver_genes.txt that are expressed that we missed? Why or why not? What complexities are we overlooking?

*Hint: Think about issues such as where different marks should be located relative to the gene (from the Enhancer paper in the prelab), whether bedtool's window command was the ideal one to use, and cell type heterogeneity*.

### Creating Pipelines and Automation of analysis (Rscripts)

In module 22, you created an Rscript that you can use for high-throughput analysis of some data

**Q7.** (5 points) Provide a copy of the Rscript that you created in your `H03_Homework-III` directory!

In [1]:
### Gene Expression Standardization Pipeline
### Author: Vidur Saigal
### Created on: 2024-12-09
### Description: This pipeline standardizes gene expression values by merging gene expression and 
### location datasets, calculating mean and standard deviation, and outputting the normalized results.

## BLOCK ZERO
## Load necessary libraries
install.packages('tidyverse', repos='http://cran.us.r-project.org')
library(tidyverse)

## BLOCK 0.5
## Obtain file paths for input files from the command line
args <- commandArgs(trailingOnly = TRUE)
if (length(args) != 2) {
  stop("Please provide two input files: gene expression file and gene location file.")
}
gene_expr_file <- args[1]  # First argument: Gene expression file
gene_loc_file <- args[2]   # Second argument: Gene location file

## BLOCK ONE
## Read input files and merge them based on 'geneid', then reorganize columns
GExpr <- read.table(file = gene_expr_file, sep = ",", header = TRUE)
Loc <- read.table(file = gene_loc_file, sep = ",", header = TRUE)
z <- left_join(GExpr, Loc, by = "geneid") %>%
      relocate(chr, pos, .after = geneid)

## BLOCK TWO
## Calculate average and standard deviation for each gene across expression data
x <- z %>%
  rowwise(geneid) %>%
  mutate(ave = mean(c_across(starts_with("GTEX")), na.rm = TRUE)) %>%
  mutate(sd = sd(c_across(starts_with("GTEX")), na.rm = TRUE)) %>%
  relocate(ave, sd, .after = pos)

## BLOCK THREE
## Standardize gene expression values for each gene
for (i in seq_along(rownames(x))) {
  for (j in 6:length(x[i, ])) {
    this_ave <- x[i, ]$ave
    this_sd <- x[i, ]$sd
    x[i, j] <- (x[i, j] - this_ave) / this_sd
  }
}

## BLOCK FOUR
## Recalculate average and standard deviation for standardized values
x <- x %>%
  rowwise(geneid) %>%
  mutate(ave_std = mean(c_across(starts_with("GTEX")), na.rm = TRUE)) %>%
  mutate(sd_std = sd(c_across(starts_with("GTEX")), na.rm = TRUE)) %>%
  relocate(ave_std, sd_std, .after = sd)

## BLOCK FIVE
## Write the output to a file with a unique name based on the input gene expression file
outfile <- paste0(gene_expr_file, ".std")
write.table(
  x,
  file = outfile,
  sep = ",",
  row.names = FALSE,
  col.names = TRUE,
  quote = FALSE
)

Installing package into ‘/opt/homebrew/lib/R/4.4/site-library’
(as ‘lib’ is unspecified)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.4     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


ERROR: Error: Please provide two input files: gene expression file and gene location file.


**Q8.** (5 points) Now, let's unlock the power of automation. We have provided you with 3 data sets (exp1, exp2, exp3):

    /data
    
Using your script, analyze each of the 3 data sets provided and generate outputs!

**Q9a** (3 points) In an R code block, read each of the 3 output files you created into R and print the contents to your notebook. 

Provide your code below:

In [2]:
# Define the prefixes for the three datasets
prefixes <- c("exp1", "exp2", "exp3")

# Loop through each prefix, read the corresponding output file, and print its contents
for (prefix in prefixes) {
  # Construct the file path for the output file
  file_path <- paste0("./data/GExp_snippet_", prefix, ".csv.std")
  
  # Read the output file into R
  data <- read.csv(file_path, header = TRUE)
  
  # Print a message indicating which file is being printed
  cat("\nContents of file:", file_path, "\n")
  
  # Print the contents of the file
  print(data)
}


Contents of file: ./data/GExp_snippet_exp1.csv.std 
     geneid chr      pos        ave        sd       ave_std sd_std   GTEX.A01
1  ENSG0001  11  1023832  1.3581406 0.6740508 -1.221245e-16      1  0.5727588
2  ENSG0002  17   199299  0.6587927 0.5591561 -1.554312e-16      1  0.4471766
3  ENSG0003  22   111238 -1.1255273 2.3377276  0.000000e+00      1  1.3713467
4  ENSG0004   1   178623 -1.3107034 0.7519454  3.330669e-17      1 -1.0207422
5  ENSG0005   2  9919267 -1.1706817 0.7299675 -1.190020e-16      1 -0.2275287
6  ENSG0006   3    17727  0.4803374 1.1899238  1.117162e-16      1  0.1247388
7  ENSG0007   4   129378  0.1127173 1.8979994  1.110223e-17      1 -1.0528440
8  ENSG0008   5  1929388  0.2793343 1.9080912  2.220446e-17      1  0.4922006
9  ENSG0009   6   884843 -0.1478188 2.3057015 -8.326673e-17      1  0.1493075
10 ENSG0010   7 27830940 -1.3712548 1.5034761 -9.645063e-17      1  2.1208943
      GTEX.A02   GTEX.A03    GTEX.A04   GTEX.A05   GTEX.A06   GTEX.A07
1  -0.54168425 -1.

**Q9b** (2 points) Using a **SINGLE** UNIX command, view the contents of all three output files. HINT: the commands `head` and `cat` can display multiple files with only one command - you might need to look up how to do this.

Provide your code below:

**Q10.** (5 points) You'll note that if you had 1000s of experiments to analyze, it would be a real pain to write out the command line for each one -- you have better things to do than that!

What could you do to save yourself from needing to do that? How would you modify the code to achieve that? (Provide a general description, but no need to write specific code)

### Automation / re-analysis Using Jupyter Notebooks

In Module 23 (Pharmcology), you will have probably noted that you could have tabulated virtually all of the above in excel. 

However, in a true high-throughput screening assay, you will have **hundreds** of plates to process. That's too much for even one human to do in excel, perfectly! 

You may also have noticed that through doing this assignment, you have written a 'generic' pipeline to process a single plate.

**Q11.** (5 points) Return to the in-class module. In order to process a different plate, called `plate2`, what would you change in the pipeline you created?

**Q12.** (5 points) In your module 23 directory, we included data from 6 additional plates. 

Process each and report here (excluding controls):
- the Z prime factor for each plate  
- the number of cells that gave lower than a **-4** normalized score, excluding controls, per plate. Note that rather than counting the results on the heatmap, you could `sum()` within the appropriate part of the heatmap table (excluding the controls, of course)

To do this, you could change the plate and re-run the markdown, and record the results in the cells below.

You will notice that in these data, you do not actually have any sample names attached to your data, e.g. what genes you actually screened.

Imagine that you were provided a file that looked like the following:

    sampleid,row,col,plate
    87234,C,3,1
    7134,C,4,1
    ...
    81672,P,22,7

i.e. a file with 2240 rows (+1 header) where each sampleids was mapped to a corresponding row and column. Note that positive and negative control columns are excluded.

**Q13.** (5 points) Imagine that you now wanted to obtain the sampleids (i.e., the gene code id!) from the a set of cells that were of interest, the 'hits' from the screen. 

For that, imagine that you had a second file which collated all of the cells across all plates which had a normalized Z-score less than -5. e.g., it looked like this:

    row,col,plate
    C,3,1
    D,5,2
    P,6,7

Describe in words the steps that would allow a computer to print out the sampleids ONLY for the entries listed in this new file. To help you, we have provided the first two steps in the process. You complete the rest!
   * Be specific in the details of what you would check for during your look-up.
   * Hint: Pretend of you had two sheets of paper, each with your lists, and you had to do this 'by hand'. What would you do, step-by-step?

**Q13 BONUS** (0 points) The following problem is optional, and will not affect your score on this homework in any way. However, if you provide a correct solution, you will get a "first Dibs" certificate for an event on the last day of class. *This will have nothing to do with your grade*.

Write code in R that implements your solution to Q13. You may use tidyverse. We have provided both files described in the question in the `q13_bonus` directory. Note that this data was randomly generated, and does not match up with data from the Pharmacology module.

### Prediction measurements and Machine Learning

**Q14.** (6 points) In the context of machine learning problems, describe what is meant by the following terms:

- Features
- Examples
- Labels


**Q15.** Imagine data from two trained models applied to a test set:

Model 1:
- Correctly labelled true positives examples: 601
- Correctly laballed true negatives examples: 999
- Actually negative examples labelled as positive: 371
- Actually positive examples labelled as negative: 29

Model 2:
- Correctly labelled true positives examples: 255
- Correctly laballed true negatives examples: 1345
- Actually negative examples labelled as positive: 25
- Actually positive examples labelled as negative: 375

For calculations, we've inputted these numbers in R for you per the following:

In [4]:
m1_tp = 601
m1_tn = 999
m1_fp = 371
m1_fn = 29

m2_tp = 255
m2_tn = 1345
m2_fp = 25
m2_fn = 375

**Q15a.** (6 points) What is the accuracy of Model 1 and Model 2, respectively?

In [5]:
# Calculate accuracy for Model 1
m1_accuracy <- (m1_tp + m1_tn) / (m1_tp + m1_tn + m1_fp + m1_fn)

# Calculate accuracy for Model 2
m2_accuracy <- (m2_tp + m2_tn) / (m2_tp + m2_tn + m2_fp + m2_fn)

# Print the results
cat("Accuracy of Model 1:", m1_accuracy, "\n")
cat("Accuracy of Model 2:", m2_accuracy, "\n")

Accuracy of Model 1: 0.8 
Accuracy of Model 2: 0.8 


**Q15b.** (6 points) Of the examples that were *lablled* positive, what proportion are correctly predicted in Model 1? Model 2?

In [6]:
# Calculate precision for Model 1
m1_precision <- m1_tp / (m1_tp + m1_fp)

# Calculate precision for Model 2
m2_precision <- m2_tp / (m2_tp + m2_fp)

# Print the results
cat("Precision of Model 1:", m1_precision, "\n")
cat("Precision of Model 2:", m2_precision, "\n")

Precision of Model 1: 0.6183128 
Precision of Model 2: 0.9107143 


**Q15c.** (6 points) Of the examples that are *actually* negative, what proportion are correctly predicted in Model 1? Model 2?

In [7]:
# Calculate specificity for Model 1
m1_specificity <- m1_tn / (m1_tn + m1_fp)

# Calculate specificity for Model 2
m2_specificity <- m2_tn / (m2_tn + m2_fp)

# Print the results
cat("Specificity of Model 1:", m1_specificity, "\n")
cat("Specificity of Model 2:", m2_specificity, "\n")

Specificity of Model 1: 0.7291971 
Specificity of Model 2: 0.9817518 


**Q15d.** (6 points) Of the examples that are *actually* positive, what proportion are correctly predicted in Model 1? Model 2?

In [8]:
# Calculate recall for Model 1
m1_recall <- m1_tp / (m1_tp + m1_fn)

# Calculate recall for Model 2
m2_recall <- m2_tp / (m2_tp + m2_fn)

# Print the results
cat("Recall of Model 1:", m1_recall, "\n")
cat("Recall of Model 2:", m2_recall, "\n")

Recall of Model 1: 0.9539683 
Recall of Model 2: 0.4047619 


**Q15e.** (8 points) Based on the above, are these two models producing equivalent performance? Why or why not? Describe a situation where application of Model 1 would be preferrable to Model 2; and conversely, a situation where application of Model 2 might be preferrable to Model 1. 